In [10]:
from src.feature_engineering.benchmark_utils import *
from src.feature_engineering.word2vec_benchmark import *
from src.data_operations.rw_utils import read_from_csv
import pandas as pd

In [6]:
tweet_train = read_from_csv("preprocess_train.csv")
tweet_train.head()

Data is read. Len of the data 13806 and columns Index(['tweet', 'sentiment'], dtype='object')


,tweet,sentiment
0,internet sadece kasar duser cocuklari,olumsuz
1,ayrica reklami kadar kotu bir reklam gormemistim,olumsuz
2,pahali,olumsuz
3,kasta internetin cekmiyor,olumsuz
4,allah belasi versin demek isterdim demicem dem...,olumsuz


In [7]:
tweet_test = read_from_csv("preprocess_test.csv")
tweet_test.head()

Data is read. Len of the data 3456 and columns Index(['tweet', 'sentiment'], dtype='object')


,tweet,sentiment
0,kadar serefsiz misiniz,olumsuz
1,burdan sesleniyorum haram olsun,olumsuz
2,hayatimda kadar kazik operator gormedim,olumsuz
3,gozumde son demlerini yasiyor hatti degisicem ...,olumsuz
4,sonra oldugunu gorup sovuyorum,olumsuz


In [8]:
tweet_test.dropna(inplace=True)
tweet_test.reset_index(drop=True,inplace=True)

tweet_train.dropna(inplace=True)
tweet_train.reset_index(drop=True,inplace=True)

x_train=tweet_train.tweet
y_train=tweet_train.sentiment.map({'olumlu':1,'olumsuz':-1,'notr':0}).values
x_test=tweet_test.tweet
y_test=tweet_test.sentiment.map({'olumlu':1,'olumsuz':-1,'notr':0}).values

In [11]:
concat = pd.concat([x_train, x_test])
corpus = labelize_tweets_ug(concat, 'all')

In [12]:
corpus_train = pd.DataFrame(x_train)['tweet'].apply(lambda x: x.split())
corpus_test = pd.DataFrame(x_test)['tweet'].apply(lambda x: x.split())

In [13]:
# min_count = corpusta kelimenin en az bulunma sayısı eğer kelime bu kadardan az geçiyorsa anlam ifade etmediği varsayılır(default=5)
# vector_size = kelimelerin ifade edileceği vektörün boyut sayısı
# window = current ve predicted word arasındaki maksimum mesafe
# sg = 0 cbow, 1 skip-gram
# negative = eğer sıfırdan büyük olursa negative sampling kullanılır 5-20 arasında olmalı
# alpha = başlangıç learning rate
# min_alpha = eğitim aşamasında learning rate linear olarak bunu düşer
# epoch = iterasyon sayısı

In [14]:
get_max_len_sentence(pd.DataFrame(concat).tweet)

The maximum length in words are : 27


In [15]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="tweet", 
                         sg=0, 
                         negative=7,
                         alpha = 0.07,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)

100%|██████████| 17222/17222 [00:00<00:00, 1439045.01it/s]


In [16]:
vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

In [17]:
classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

LogisticRegression:
accuracy: 0.622969837587007
precision: 0.6186956206610763
recall: 0.6086989221132949
              precision    recall  f1-score   support

          -1       0.64      0.75      0.69      1373
           0       0.62      0.55      0.58      1160
           1       0.60      0.53      0.56       915

    accuracy                           0.62      3448
   macro avg       0.62      0.61      0.61      3448
weighted avg       0.62      0.62      0.62      3448

RandomForest:
accuracy: 0.6337006960556845
precision: 0.6424744869955755
recall: 0.6112209053139425
              precision    recall  f1-score   support

          -1       0.62      0.81      0.70      1373
           0       0.65      0.55      0.60      1160
           1       0.67      0.47      0.55       915

    accuracy                           0.63      3448
   macro avg       0.64      0.61      0.62      3448
weighted avg       0.64      0.63      0.63      3448

SVM:
accuracy: 0.6540023201856149